In [ ]:
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  stocks:
    - AAPL
    - MSFT
    - GOOGL
  start_date: "2023-01-01"
  end_date: "2023-09-01"
  output_directory: artifacts/data


In [1]:
import os
os.chdir("../")
%pwd

'd:\\projects\\pet\\MLFlow\\Feature_Eng'

In [2]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass
class DataIngestionConfig:
  root_dir: Path
  stocks: List[str]
  start_date: str
  end_date: str
  output_directory: Path

from mlFeature.constants import *
from mlFeature.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            stocks=config.stocks,
            start_date=config.start_date,
            end_date=config.end_date,
            output_directory=config.output_directory
        )
        return data_ingestion_config
    
    


In [4]:
import os
import urllib.request as request
import zipfile
from mlFeature import logger
from mlFeature.utils.common import get_size
import yfinance as yf

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_yahoo_finance_data(self):
        if not os.path.exists(self.config.output_directory):
            os.mkdir(self.config.output_directory)
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.output_directory))}")
            # Download data for each stock in the configuration
        for stock in self.config.stocks:
                data = yf.download(stock, start=self.config.start_date, end=self.config.end_date)
                # Save data to the specified directory
                data.to_csv(f"{self.config.output_directory}/{stock}.csv")
                logger.info(f"{stock} download! Data info: {data.shape} rows x {data.shape[1]} columns")
        
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_yahoo_finance_data()
except Exception as e:
    logger.error(f"Error: {str(e)}")
    raise e


[2023-09-14 00:47:11,046: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-14 00:47:11,048: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-14 00:47:11,050: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-14 00:47:11,052: INFO: common: created directory at: artifacts]
[2023-09-14 00:47:11,054: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-14 00:47:11,055: INFO: 1552191217: File already exists of size: ~ 0 KB]
[*********************100%%**********************]  1 of 1 completed
[2023-09-14 00:47:12,051: INFO: 1552191217: AAPL download! Data info: (167, 6) rows x 6 columns]
[*********************100%%**********************]  1 of 1 completed
[2023-09-14 00:47:12,937: INFO: 1552191217: MSFT download! Data info: (167, 6) rows x 6 columns]
[*********************100%%**********************]  1 of 1 completed
[2023-09-14 00:47:13,758: INFO: 1552191217: GOOGL download! Data info: (167, 6) rows x 6 columns]
